In [190]:
# Instalacion de dependencias
pip install sqlalchemy psycopg2-binary
pip install pyarrow

SyntaxError: invalid syntax (1181715430.py, line 2)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Verificacion de pandas
pd.__version__

'1.5.1'

## Explorando Pandas
[1] [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) Wes McKinney

### Series

A **Series** is a single vector of data (like a NumPy 1-d array) with an *index* that labels each element in the vector.

In [3]:
counts = pd.Series([632, 1638, 569, 115])
counts

0     632
1    1638
2     569
3     115
dtype: int64

In [4]:
counts.values

array([ 632, 1638,  569,  115], dtype=int64)

If an <b>index</b> is not specified, a default sequence of integers is assigned as the index. A NumPy array comprises the values of the `Series`, while the index is a pandas `Index` object.

In [5]:
counts.index

RangeIndex(start=0, stop=4, step=1)

We can assign meaningful labels to the index, if they are available:

In [6]:
bacteria = pd.Series([632, 1638, 569, 115],
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

In [7]:
bacteria['Actinobacteria'] #dictionary style

569

In [8]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

### DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

In [9]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria',
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
data

,value,patient,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,1130,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Notice the `DataFrame` is sorted by column name. We can change the order by indexing them in the order we desire:

In [10]:
data[['phylum','value','patient']]

,phylum,value,patient
0,Firmicutes,632,1
1,Proteobacteria,1638,1
2,Actinobacteria,569,1
3,Bacteroidetes,115,1
4,Firmicutes,433,2
5,Proteobacteria,1130,2
6,Actinobacteria,754,2
7,Bacteroidetes,555,2


A `DataFrame` has a second index:

In [11]:
data.index

RangeIndex(start=0, stop=8, step=1)

### Exercise 1

Find the size of the first dimension of `data`.

In [12]:
# write the code here
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria',
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})

print("Size: ", data.size)
print("Shape: ", data.shape)
print("ndim: ", data.ndim)

Size:  24
Shape:  (8, 3)
ndim:  2


Create a DataFrame from a list of dicts. Each dictionary have to contain the same properties(fields), for this exercise consider: 'name', 'address', 'age'

In [13]:
# write the code here
datExample = [ {'Name':'Paola Vilaseca', 'Address':'Casto Rojas 1528', 'Age':'23'}]
dataFrame = pd.DataFrame.from_dict(datExample)
dataFrame


,Name,Address,Age
0,Paola Vilaseca,Casto Rojas 1528,23


From the `data` table above, return all rows for which the phylum name ends in "bacteria" and the value is greater than 1000.
Hint: apply method might come handy in here.

In [14]:
# write your code here
data[data.phylum.str.endswith('bacteria')] 
data[data.value > 1000] 

,value,patient,phylum
1,1638,1,Proteobacteria
5,1130,2,Proteobacteria


## Conectando a PostgreSQL

In [15]:
from sqlalchemy import create_engine

In [16]:
POSTGRES_USERNAME = "root"
POSTGRES_PASSWORD = "root"
POSTGRES_URL = "192.168.56.1" #ip de la laptop
POSTGRES_PORT = 5432
POSTGRES_DATABASE = "ny_taxi"
engine =  create_engine(f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_URL}:{POSTGRES_PORT}/{POSTGRES_DATABASE}")

In [17]:
# verificar la conexion
engine.connect()

In [18]:
query = """
SELECT 'abc' as string;
"""
pd.read_sql(query, con=engine)

,string
0,abc


In [19]:
# tablas presentes en el database
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
    schemaname != 'information_schema';
"""
pd.read_sql(query, con=engine)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,yellow_taxi_data_january,root,None,True,False,False,False


## Dataset

In [20]:
! cd dataset/ && wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

'wget' is not recognized as an internal or external command,
operable program or batch file.


El dataset que usaremos es Yellow Taxi Trip Records. Usaremos data del mes de enero del 2022. Mas informacion en https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page.
El dataset se encuentra en format PARQUET. Para poder leerlo necesitaremos importar las siguientes librerias.

In [21]:
import pyarrow.parquet as pq

In [22]:
dataset_fname = 'dataset/yellow_tripdata_2022-01.parquet'
trips = pq.read_table(dataset_fname)

Pasamos a Pandas

In [23]:
df = trips.to_pandas()

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Ingestando el dataset en PostgreSQL

Podemos usar el metodo to_sql para pasar el dataset a la base de datos relacional.
chunksize: permite definir el numero de files que van a ser ingestadas al mismo tiempo. Por defecto, se ingesta todas las filas al mismo tiempo. Es importante definir un limite cuando se ingesta un dataset con un numero de files considerablemente grande.

In [25]:
table_name = "yellow_taxi_data_january"

In [26]:
df.to_sql(name=table_name, con=engine, if_exists='replace', chunksize=100000)

24931

In [27]:
query = f"""
SELECT *
FROM {table_name}
WHERE "VendorID" = 1
LIMIT 10
"""
pd.read_sql(query, con=engine)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
3,16,1,2022-01-01 00:33:52,2022-01-01 00:47:28,3.0,4.2,1.0,N,148,141,1,14.0,2.5,0.5,3.45,0.00,0.3,20.75,2.5,0.0
4,17,1,2022-01-01 00:53:54,2022-01-01 01:05:20,2.0,2.2,1.0,N,237,107,1,9.5,2.5,0.5,2.55,0.00,0.3,15.35,2.5,0.0
5,18,1,2022-01-01 00:00:44,2022-01-01 00:05:29,1.0,0.2,1.0,N,7,7,2,5.0,0.5,0.5,0.00,0.00,0.3,6.30,0.0,0.0
6,19,1,2022-01-01 00:35:50,2022-01-01 00:48:33,2.0,3.9,1.0,N,107,263,1,13.0,3.0,0.5,3.35,0.00,0.3,20.15,2.5,0.0
7,20,1,2022-01-01 00:49:14,2022-01-01 00:58:29,2.0,3.2,1.0,N,263,107,1,11.0,3.0,0.5,2.95,0.00,0.3,17.75,2.5,0.0
8,21,1,2022-01-01 00:06:10,2022-01-01 00:08:58,1.0,0.0,1.0,N,161,161,4,2.5,3.0,0.5,0.00,0.00,0.3,6.30,2.5,0.0
9,22,1,2022-01-01 00:09:01,2022-01-01 00:17:15,1.0,1.2,1.0,N,161,43,1,7.0,3.0,0.5,2.15,0.00,0.3,12.95,2.5,0.0


In [28]:
query = f"""
SELECT *
FROM {table_name}
WHERE "VendorID" = 1
LIMIT 10
"""
pd.read_sql(query, con=engine)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
3,16,1,2022-01-01 00:33:52,2022-01-01 00:47:28,3.0,4.2,1.0,N,148,141,1,14.0,2.5,0.5,3.45,0.00,0.3,20.75,2.5,0.0
4,17,1,2022-01-01 00:53:54,2022-01-01 01:05:20,2.0,2.2,1.0,N,237,107,1,9.5,2.5,0.5,2.55,0.00,0.3,15.35,2.5,0.0
5,18,1,2022-01-01 00:00:44,2022-01-01 00:05:29,1.0,0.2,1.0,N,7,7,2,5.0,0.5,0.5,0.00,0.00,0.3,6.30,0.0,0.0
6,19,1,2022-01-01 00:35:50,2022-01-01 00:48:33,2.0,3.9,1.0,N,107,263,1,13.0,3.0,0.5,3.35,0.00,0.3,20.15,2.5,0.0
7,20,1,2022-01-01 00:49:14,2022-01-01 00:58:29,2.0,3.2,1.0,N,263,107,1,11.0,3.0,0.5,2.95,0.00,0.3,17.75,2.5,0.0
8,21,1,2022-01-01 00:06:10,2022-01-01 00:08:58,1.0,0.0,1.0,N,161,161,4,2.5,3.0,0.5,0.00,0.00,0.3,6.30,2.5,0.0
9,22,1,2022-01-01 00:09:01,2022-01-01 00:17:15,1.0,1.2,1.0,N,161,43,1,7.0,3.0,0.5,2.15,0.00,0.3,12.95,2.5,0.0


### Exercise 2

Cuantos tipos (distintos) de VendorID hay?.

In [34]:
# write your code here
query = f"""
SELECT COUNT(DISTINCT "VendorID")

FROM {table_name}
"""
pd.read_sql(query, con=engine)

,count
0,4


Mostrar informacion del viaje con mas pasajeros en Enero de 2022.

In [32]:
# write your code here
query = f"""

select *
from {table_name}
where passenger_count = (
    select max(passenger_count)
    from {table_name}
)

"""
pd.read_sql(query, con=engine)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1183241,2,2022-01-16 23:43:38,2022-01-16 23:46:00,9.0,0.12,5.0,N,141,141,2,9.3,0.0,0.5,0.0,0.0,0.3,12.60,2.5,0.00
1,685853,2,2022-01-10 18:06:31,2022-01-10 18:06:39,9.0,0.00,5.0,N,132,132,1,96.5,0.0,0.0,10.0,0.0,0.3,108.05,0.0,1.25
2,779193,2,2022-01-12 07:11:53,2022-01-12 08:12:47,9.0,28.40,5.0,N,132,265,2,90.0,0.0,0.0,0.0,0.0,0.3,91.55,0.0,1.25


In [31]:
""

''